

> Import libraries to use



In [245]:
import numpy as np

>  # Introduction to numpy (Skip if you already are familiar)

>> Creating a 1D array

In [246]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


>> Creating a 2D array


In [247]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


>> Creating an array full of zeros


In [248]:
a = np.zeros(shape=(10))
print(a)
a = np.zeros(shape=(5,2))
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


>> Infinity in numpy

In [249]:
print(np.inf)

inf


>> Max and Argmax

In [250]:
a = np.array([2,1,4,3])
print(np.max(a))
print(np.argmax(a))

4
2


>> From list to Numpy

In [251]:
l = [1,2,3,4]
print(l)
print(np.asarray(l))

[1, 2, 3, 4]
[1 2 3 4]


>> Random in numpy

In [252]:
# Array of Random integers ranging from 1 to 10 (with any size you want)
a = np.random.randint(low=1, high=10, size=(5,2))
print(a)

# Array of random elements of a list with any size you want
a = np.random.choice([0,1,2], size=(2,))

[[5 1]
 [3 8]
 [6 3]
 [3 4]
 [1 5]]


>> Shapes in numpy

In [253]:
a = np.random.randint(low=1, high=5, size=(4,2))
print(a.shape)
print(a)

# Reshape a to a vector of shape = (8,1)
a = a.reshape((8,1))
print(a.shape)
print(a)

(4, 2)
[[3 4]
 [3 4]
 [4 4]
 [4 2]]
(8, 1)
[[3]
 [4]
 [3]
 [4]
 [4]
 [4]
 [4]
 [2]]


# Pre-defined utilities

In [254]:

int_to_char = {
    0 : 'u',
    1 : 'r',
    2 : 'd',
    3 : 'l'
}

policy_one_step_look_ahead = {
    0 : [-1,0],
    1 : [0,1],
    2 : [1,0],
    3 : [0,-1]
}

def policy_int_to_char(pi,n):


    pi_char = []
    for i in range(n):
        for j in range(n):
            if (i, j) == (0, 0) or (i, j) == (n - 1, n - 1):
                pi_char.append(' ')  # Represent terminal states with a space
                continue

            # Find the action with probability 1 (assuming deterministic policy)
            action = np.argmax(pi[i, j])

            # Use the action index to get the corresponding character
            pi_char.append(int_to_char[action])

        pi_char.append('\n')  # Add a newline character at the end of each row

    return ''.join(pi_char)

# 1- Policy evaluation

In [255]:
def policy_evaluation(n,pi,v,Gamma,threshhold, max_iterations = 10000):
      """
        This function should return the value function that follows the policy pi.
        Use the stopping criteria given in the problem statement.
      """
      reward = -1
      iteration = 0
      while True:
            delta = 0
            iteration = iteration + 1
    
            for i in range(n):
                for j in range(n):
                    if (i, j) == (0, 0) or (i, j) == (n-1, n-1):
                        continue  # Skip terminal states
    
                    v_old = v[i, j]
                    v_new = 0
    
                    for action in range(4):
                        i_prime = i + policy_one_step_look_ahead[action][0]
                        j_prime = j + policy_one_step_look_ahead[action][1]
    
                        if not (0 <= i_prime < n and 0 <= j_prime < n):
                            i_prime, j_prime = i, j
    
                        v_new += pi[i, j, action] * (reward + Gamma * v[i_prime, j_prime])
                    v[i, j] = v_new
                    delta = max(delta, abs(v_old - v_new))
            if delta < threshhold or iteration == max_iterations:
                break
    
      return v
    
  
  


# 2- Policy improvement

In [256]:
def policy_improvement(n,pi,v,Gamma):
      """
        This function should return the new policy by acting in a greedy manner.
        The function should return as well a flag indicating if the output policy
        is the same as the input policy.
    
        Example:
          return new_pi, True if new_pi = pi for all states
          else return new_pi, False
      """
      new_pi = np.zeros((n, n, 4))
      reward = -1
      policy_stable = True
    
      for i in range(n):
            for j in range(n):
                if (i, j) == (0, 0) or (i, j) == (n-1, n-1):
                    continue
    
                q_values = np.zeros(4)
                for action in range(4):
                    i_prime = i + policy_one_step_look_ahead[action][0]
                    j_prime = j + policy_one_step_look_ahead[action][1]
    
                    if i_prime < 0 or i_prime >= n or j_prime < 0 or j_prime >= n:
                        i_prime, j_prime = i, j
    
                    q_values[action] = reward + Gamma * v[i_prime, j_prime]
    
                max_q = np.max(q_values)
                best_actions = [action for action in range(4) if q_values[action] == max_q]
    
                for action in range(4):
                    if action in best_actions:
                        new_pi[i, j, action] = 1 / len(best_actions)
                    else:
                        new_pi[i, j, action] = 0
    
                if not np.array_equal(new_pi[i, j], pi[i, j]):
                    policy_stable = False
    
      return new_pi, policy_stable
  
  

# 3- Policy Initialization

In [257]:
def policy_initialization(n):
  """
    This function should return the initial random policy for all states.
  """
  pi = np.zeros((n, n, len(policy_one_step_look_ahead)))

  for i in range(n):
      for j in range(n):
          if (i, j) == (0, 0) or (i, j) == (n-1, n-1):
              pi[i, j] = [0, 0, 0, 0]
          else:
              random_action = np.random.choice(len(policy_one_step_look_ahead))
              pi[i, j, random_action] = 1

  return pi
  

# 4- Policy Iteration algorithm

In [258]:
def policy_iteration(n,Gamma,threshhold):

    pi = policy_initialization(n=n)

    v = np.zeros(shape=(n,n))

    while True:

        v = policy_evaluation(n=n,v=v,pi=pi,threshhold=threshhold,Gamma=Gamma)

        pi , pi_stable = policy_improvement(n=n,pi=pi,v=v,Gamma=Gamma)

        if pi_stable:

            break

    return pi , v

# Main Code to Test

In [259]:
n = 4

Gamma = [0.8,0.9,1]

threshhold = 1e-4

for _gamma in Gamma:

    pi , v = policy_iteration(n=n,Gamma=_gamma,threshhold=threshhold)

    pi_char = policy_int_to_char(n=n,pi=pi)

    print()
    print("Gamma = ",_gamma)

    print()

    print(pi_char)

    print()
    print()

    print(v)



Gamma =  0.8

 lld
uuud
uurd
urr 



[[ 0.   -1.   -1.8  -2.44]
 [-1.   -1.8  -2.44 -1.8 ]
 [-1.8  -2.44 -1.8  -1.  ]
 [-2.44 -1.8  -1.    0.  ]]

Gamma =  0.9

 lld
uuud
uurd
urr 



[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]

Gamma =  1

 lld
uuud
uurd
urr 



[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
